# Trakking Road Mapping Sample

Note that the sample only works with ipyleaflet extension installed.
For details, refer [the official documentation](https://ipyleaflet.readthedocs.io/en/latest/installation.html)

In [1]:
import ipyleaflet as ipl

# Initialize map object
m = ipl.Map(zoom=11, center=(36.382832947767206, 137.49234340153637))
m.layout.width = '70%'
m.layout.height = '500px'
m

Map(center=[36.382832947767206, 137.49234340153637], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [2]:
# The behavior is a bit unstable
# Wait a little bit and retry if it doesn't work
(south, east,), (north, west,) = m.bounds
south, east, north, west

(36.24427318493909, 137.35244750976565, 36.52067329034796, 137.63259887695315)

In [3]:
%%time
import overpy
api = overpy.Overpass()
query = f"""
    way
    ["highway"]
    ["area"!~"yes"]
    ["highway"!~"cycleway|motor|proposed|construction|abandoned|platform|raceway"]
    ["foot"!~"no"]
    ["service"!~"private"]
    ({south}, {east}, {north}, {west});
    out;
"""
print(f"[*] Query: {query}")
result = api.query(query)
print("[*] Found nodes:", len(result.nodes))
print("[*] Found ways:", len(result.ways))

[*] Query: 
    way
    ["highway"]
    ["area"!~"yes"]
    ["highway"!~"cycleway|motor|proposed|construction|abandoned|platform|raceway"]
    ["foot"!~"no"]
    ["service"!~"private"]
    (36.24427318493909, 137.35244750976565, 36.52067329034796, 137.63259887695315);
    out;

[*] Found nodes: 0
[*] Found ways: 1816
Wall time: 3.47 s


In [4]:
%%time
import ipyleaflet as ipl
from tqdm.notebook import tqdm

# if only the nodes in between is important
ways = result.ways
node_ids = sorted(
    set(
        w._node_ids[0] for w in ways
    ).union(
        w._node_ids[-1] for w in ways
    )
)
query = "[out:json];\n" + "\n".join(
    [f"node({node_id});out;" for node_id in node_ids]
)
print(f"[*] Querying {len(node_ids)} nodes")
result_nids = api.query(query)
print("[*] Done")
nid2latlon = {n.id: (float(n.lat), float(n.lon)) for n in result_nids.nodes}
del node_ids, query, result_nids

# If all the nodes are important
# for way in tqdm(ways):
#     way.get_nodes(resolve_missing=True)

line = ipl.Polyline(
    locations=[
        [
            nid2latlon[n_id]
            for n_id in way._node_ids
            if n_id in nid2latlon
        ]
        for way in ways
    ],
    color='RED',
    fill_opacity=0.0,
    weight=1,
)
m.add_layer(line)
m

[*] Querying 1814 nodes


Map(bottom=205440.0, center=[36.382832947767206, 137.49234340153637], controls=(ZoomControl(options=['position…

[*] Done
Wall time: 8.2 s
